In [1]:
using JLD
include("gyncycle.jl");

In [2]:
SIGMA_RHO = 0.3
SIGMA_Y0 = 10
SIGMA_PARMS = 10
SIGMA_PROPOSAL = 0.01
adapt = :all

parms, y0 = loadparms()
samplingnodes = [:sparms, :y0]
data = loadpfizer()
persons = 1:3

proposalvariance = eye(length(SAMPLEPARMS)+length(y0)) * log(1+(SIGMA_PROPOSAL^2));

In [ ]:
mcs = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, [AMM(samplingnodes, proposalvariance, adapt=:all)])
    
    push!(mcs, mcmc(m, inp, ini, 1000, burnin=0, verbose = false, thin = 1))
end

#samplelength(mc) = max(floor(Int, size(mc, 1)/5), 2_000)
jldname = "out/p$(length(persons))-adapt$adapt-r$SIGMA_RHO-sy$SIGMA_Y0-spar$SIGMA_PARMS-spro$SIGMA_PROPOSAL-2truncflatprior.jld"

while size(mcs[1],1) < 1_000_000
    @time save(jldname, "mc" , [mc.value for mc in mcs])    
    @time for p in 1:length(persons)
        mcs[p] = mcmc(mcs[p], 1_000, verbose=false)
    end
    println(size(mcs[1]))
end